In [2]:
import datetime
import sqlite3 as sql 
import requests
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import json 
import networkx
import os
from nlp_helper import *
from pdfminer.high_level import extract_text
from config_new import config
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...


[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vaugh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-d

True

In [3]:
con = sql.connect(config['database'])
cur = con.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS Front_Page_text (id INTEGER PRIMARY KEY AUTOINCREMENT, front_page_text TEXT, date_dd_mm_yyyy TEXT, slug TEXT)""")
cur.execute("""CREATE TABLE IF NOT EXISTS Front_Page_vader (front_page_text_id INTEGER, sediment_neg REAL, sediment_neu REAL, sediment_pos REAL, sediment_comp REAL)""")
cur.execute("""CREATE TABLE IF NOT EXISTS Front_Page_bag (front_page_text_id INTEGER, bag_a_words_json BLOB, Tfidf_json BLOB, phrase_list BLOB)""")
cur.execute("""CREATE VIEW IF NOT EXISTS Front_Page_Report
               AS
               SELECT a.slug, a.date_dd_mm_yyyy, b.sediment_comp, b.sediment_neg, b.sediment_neu, b.sediment_pos, c.bag_a_words_json, c.Tfidf_json
                FROM Front_Page_text a
                JOIN Front_Page_vader b ON a.id = b.front_page_text_id
                JOIN Front_Page_bag c ON a.id = c.front_page_text_id""")
cur.execute("""CREATE TABLE IF NOT EXISTS Front_Page_nea (front_page_text_id INT, nea_text TEXT, start_char TEXT, end_char TEXT, label TEXT)""") 
con.commit()

In [3]:
con = sql.connect(config['database'])
day = datetime.datetime.today()
cur = con.cursor()
slugs = pd.read_sql_query("""SELECT * FROM Front_Page_slug""",con)
slugs = slugs['slug'].tolist()
for slug in slugs:
    url_temp =  f'https://cdn.freedomforum.org/dfp/pdf{day.day}/{slug}.pdf'
    r = requests.get(url_temp)
    if r.status_code == 200:
        with open(f'{slug}.pdf', 'wb') as f:
            f.write(r.content)
        text = extract_text(f'{slug}.pdf')
        os.remove(f'{slug}.pdf')
        cur.execute(f"""INSERT INTO Front_Page_text (front_page_text, day, slug) VALUES ('{text.replace("'","''")}', '{day.day}-{day.month}-{day.year}', '{slug}')""")
        con.commit()
con.close()

Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


...

Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start and end byte have different lengths.


empty vocabulary; perhaps the documents only contain stop words
 
empty vocabulary; perhaps the documents only contain stop words
 
empty vocabulary; perhaps the documents only contain stop words
 
empty vocabulary; perhaps the documents only contain stop words
 


empty vocabulary; perhaps the documents only contain stop words
 


' '